In [0]:
import pandas as pd 
ds=pd.read_csv("/content/drive/My Drive/patient.csv")

In [0]:
missing=((ds.isnull().sum())/ds.shape[0])*100

In [0]:
missing

HR                   9.882619
O2Sat               13.061119
Temp                66.162697
SBP                 14.576958
MAP                 12.451279
DBP                 31.345887
Resp                15.354559
EtCO2               96.286843
BaseExcess          94.579020
HCO3                95.810618
FiO2                91.665754
pH                  93.069688
PaCO2               94.440121
SaO2                96.549372
AST                 98.377604
BUN                 93.134434
Alkalinephos        98.393194
Calcium             94.116067
Chloride            95.460279
Creatinine          93.904433
Bilirubin_direct    99.807371
Glucose             82.894325
Lactate             97.329872
Magnesium           93.689578
Phosphate           95.986303
Potassium           90.689082
Bilirubin_total     98.509158
TroponinI           99.047745
Hct                 91.145979
Hgb                 92.617558
PTT                 97.055875
WBC                 93.593199
Fibrinogen          99.340167
Platelets 

In [0]:
selected_cols = list(missing[missing < 90].index)
selected_cols

['HR',
 'O2Sat',
 'Temp',
 'SBP',
 'MAP',
 'DBP',
 'Resp',
 'Glucose',
 'Age',
 'Gender',
 'Unit1',
 'Unit2',
 'HospAdmTime',
 'ICULOS',
 'SepsisLabel',
 'Patient_Id']

In [0]:
filter_ds=ds[selected_cols]

In [0]:
filter_ds

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,Glucose,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,Patient_Id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.33,1,1.0,0.0,-663.55,1,0,0
1,106.0,98.0,NaN,141.0,102.33,NaN,24.0,NaN,66.33,1,1.0,0.0,-663.55,2,0,0
2,109.0,94.0,NaN,144.0,82.00,NaN,23.0,NaN,66.33,1,1.0,0.0,-663.55,3,0,0
3,102.0,94.0,36.89,124.0,78.67,NaN,22.0,NaN,66.33,1,1.0,0.0,-663.55,4,0,0
4,95.0,97.0,NaN,120.0,73.33,NaN,21.0,NaN,66.33,1,1.0,0.0,-663.55,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552205,88.0,97.0,NaN,108.0,56.00,39.0,21.0,NaN,72.00,1,1.0,0.0,-40.63,50,0,40335
1552206,87.5,94.5,36.50,121.5,66.00,49.0,21.0,NaN,72.00,1,1.0,0.0,-40.63,51,0,40335
1552207,88.0,98.0,NaN,124.0,67.00,49.0,18.0,NaN,72.00,1,1.0,0.0,-40.63,52,0,40335
1552208,74.0,97.0,NaN,111.0,83.00,61.0,20.0,NaN,72.00,1,1.0,0.0,-40.63,53,0,40335


In [0]:
patientid=filter_ds['Patient_Id'].values

In [0]:
filter_ds=filter_ds.ffill(axis='rows')
filter_ds=filter_ds.bfill(axis='rows')

In [0]:
ff=filter_ds.drop('SepsisLabel',axis=1)

In [0]:
ff=ff.drop('Patient_Id',axis=1)

In [0]:
ff=ff.drop('HospAdmTime',axis=1)

In [0]:
ff

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,Glucose,Age,Gender,Unit1,Unit2,ICULOS
0,106.0,98.0,36.89,141.0,102.33,55.0,24.0,69.0,66.33,1,1.0,0.0,1
1,106.0,98.0,36.89,141.0,102.33,55.0,24.0,69.0,66.33,1,1.0,0.0,2
2,109.0,94.0,36.89,144.0,82.00,55.0,23.0,69.0,66.33,1,1.0,0.0,3
3,102.0,94.0,36.89,124.0,78.67,55.0,22.0,69.0,66.33,1,1.0,0.0,4
4,95.0,97.0,36.89,120.0,73.33,55.0,21.0,69.0,66.33,1,1.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552205,88.0,97.0,36.80,108.0,56.00,39.0,21.0,172.0,72.00,1,1.0,0.0,50
1552206,87.5,94.5,36.50,121.5,66.00,49.0,21.0,172.0,72.00,1,1.0,0.0,51
1552207,88.0,98.0,36.50,124.0,67.00,49.0,18.0,172.0,72.00,1,1.0,0.0,52
1552208,74.0,97.0,36.50,111.0,83.00,61.0,20.0,172.0,72.00,1,1.0,0.0,53


In [0]:
featurearr=ff.values

In [0]:
c=0
index=0
y=[]
x=[]
seq=[]
for i in range(len(patientid)):
    if i==0:
        seq+=featurearr[index].tolist()
        index+=1
        continue
    if patientid[i]!=patientid[i-1]:
        x.append(seq)
        y.append(filter_ds['SepsisLabel'].iloc[i-1])
        seq=[]
    seq+=featurearr[index].tolist()
    index+=1
x.append(seq)
y.append(filter_ds['SepsisLabel'].iloc[i-1])

In [0]:
len(x),len(y)

(40336, 40336)

In [0]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
np.random.seed(7)

Using TensorFlow backend.


In [0]:
x=np.array(x)
y=np.array(y)

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=9)

In [0]:
max_length = 4368
x_train = sequence.pad_sequences(x_train, maxlen=max_length)
x_test = sequence.pad_sequences(x_test, maxlen=max_length)

In [0]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(40336,embedding_vecor_length, input_length=max_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
#model.add(Dense(2, activation='sigmoid'))
#model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=64)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4368, 32)          1290752   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 1,344,053
Trainable params: 1,344,053
Non-trainable params: 0
_________________________________________________________________
None



Train on 24201 samples, validate on 16135 samples
Epoch 1/3





24201/24201 [==============================] - 2649s 109ms/step - loss: 0.2336 - acc: 0.9291 - val_loss: 0.1801 - val_acc: 0.9425
Epoch 2/3
24201/24201 [==============================] - 2643s 109ms/ste

In [0]:
ypred=model.predict(x_test)

In [0]:
print(ypred)

[[0.01832333]
 [0.14663398]
 [0.01933837]
 ...
 [0.04526544]
 [0.00948569]
 [0.01830082]]


In [0]:
y_pred=[]
for i in ypred:
  if(i<0.5):
    y_pred.append(0)
  else:
    y_pred.append(1)

In [0]:
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [0]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[14843,   134],
       [  640,   518]])